# 三角形最大数字和

[CSDN](https://blog.csdn.net/baidu_28312631/article/details/47418773)

在上面的数字三角形中寻找一条从顶部到底边的路径，使得路径上所经过的数字之和最大。路径上的每一步都只能往左下或 右下走。只需要求出这个最大和即可，不必给出具体路径。 三角形的行数大于1小于等于100，数字为 0 - 99
输入格式：

    5      //表示三角形的行数    接下来输入三角形

    7

    3   8

    8   1   0

    2   7   4   4

    4   5   2   6   5

要求输出最大和

## 递归方法

### 标准递归方式

In [110]:
import numpy as np

num=10
tri_mat=np.random.randint(0,100,(10,10)) 
for l in range(num):
    for m in range(num):
       if m>l:
        tri_mat[l,m]=0
print(tri_mat)

[[74  0  0  0  0  0  0  0  0  0]
 [ 6 91  0  0  0  0  0  0  0  0]
 [91 13 17  0  0  0  0  0  0  0]
 [ 1 48 56  6  0  0  0  0  0  0]
 [ 9 61 53 96 15  0  0  0  0  0]
 [51 12 43 16 31 74  0  0  0  0]
 [92 57 63 52 20 51 24  0  0  0]
 [78 58 25 59 23 86  8 89  0  0]
 [51 71 36 83  6 61 50 73  8  0]
 [17  5 93  4 15  8 73 60 63 97]]


首先用递归的方法来做，递归的思想就是我们选择第n次时候，需要用到第n+1次的结果，注意这里是倒推的，因为我们定义f(n,m)为从f(n,m)出发到达所经过的最大数字，用v(n,m)表示具体的路径上面的数字
$$
f(n,m)=max(f(n+1,m),f(n+1,m+1))+v(n,m)
$$

这样我们可以写一个简单的递归函数来表述

In [41]:
def fun_recursion(n,m,tri_mat):
    N=tri_mat.shape[0]
    if n==N-1:
        return tri_mat[n,m]
    else:
        return max(fun_recursion(n+1,m,tri_mat),fun_recursion(n+1,m+1,tri_mat))+tri_mat[n,m]

fun_recursion(1,1,tri_mat)

735

### 正向递归可以吗？
我们可以看见，直接求解出来了值，我们可以不可以正着来呢，就是我从第一步出发走到下一步，只求下一步最优的解，这样的式子应该是
$$
f(n,m)=max(f(n-1,m),f(n-1,m+1))+v(n,m)
$$
即我们每次都走最优的那条路径往下走

In [42]:
def fun_recursion_v2(n,m,tri_mat):
    if n==0:
        return tri_mat[n,m]
    else:
        return max(fun_recursion_v2(n-1,m,tri_mat),fun_recursion_v2(n-1,m-1,tri_mat))+tri_mat[n,m]
sum_last=np.zeros(num)
for l in range(num):
    sum_last[l]=fun_recursion_v2(num-1,l,tri_mat)
print(sum_last)
np.max(sum_last)

[483. 541. 611. 722. 770. 739. 665. 594. 623. 542.]


770.0

### 枚举算法？

答案并不一样，我们需要用穷举的方法来验证。即我们从每一条路径出发，每次层有两种选择，我们把所有层的结果记录下来。然后找出最大值。我们第一步有两种选择方法，第二步有$2\times2^2$，第三步有$2\times2^2\times 2^3$,第N-1步有$2^(1+2+3+..N-1)=2^(n(n-1)/2)$种方法！,这其实很容易内存爆掉，但是我们可以试试小的数组

In [131]:
import numpy as np

num=10
tri_mat=np.array(np.random.randint(0,100,(10,10)),dtype=float)
for l in range(num):
    for m in range(num):
       if m>l:
        tri_mat[l,m]=np.inf
print(tri_mat)

[[91. inf inf inf inf inf inf inf inf inf]
 [61. 77. inf inf inf inf inf inf inf inf]
 [10.  2. 72. inf inf inf inf inf inf inf]
 [68. 44. 83. 62. inf inf inf inf inf inf]
 [80. 66. 27. 22. 96. inf inf inf inf inf]
 [34. 64. 61. 59.  6.  0. inf inf inf inf]
 [ 2. 78. 56. 40. 53. 50. 85. inf inf inf]
 [44. 64. 39.  1. 78. 85.  3. 66. inf inf]
 [23. 34. 93. 26. 42.  0. 46. 13. 90. inf]
 [46. 10. 48. 60. 33.  1. 53. 99.  3.  1.]]


In [ ]:
sort_mat = np.zeros((num,num),dtype=int)
for l in range(num):
    for m in range(l+1):
        if l==0:
            sort_mat[l,0]=1
        elif l==1:
            sort_mat[l,0]=1
            sort_mat[l,1]=1
        else:
            if m==0 or m==l:
                sort_mat[l,m]=1
            else:
                sort_mat[l,m]=sort_mat[l-1,m-1]+sort_mat[l-1,m]
print(sort_mat)

In [ ]:
sort_mat_sum=np.zeros((num,num),dtype=int)
for l in range(num):
    for m in range(l+1):
        sort_mat_sum[l,m]=np.sum(sort_mat[l,0:m+1])
print(sort_mat_sum)

我们这里来进行一下规划，接下里我会用两个矩阵，分别存储上一次和这一次的路径值，然后进行更新

In [139]:
steps_I=np.ones((num,2**(num-1)))*np.inf
steps_II=np.ones((num,2**(num-1)))*np.inf
det_steps=np.ones((num,2**(num-1)))*np.inf
for l in range(num):
    for m in range(l+1):
        if l==0:
            steps_I[m,0]=tri_mat[l,0]
            steps_II[:,:]=steps_I[:,:]
        elif l==1:
            steps_I[m,0]=tri_mat[m,0]+tri_mat[0,0]
            steps_II[:,:]=steps_I[:,:]
        else:
            if m==0:
                steps_II[m,0]=steps_I[m,0]+tri_mat[l,0]
            elif m==l:
                print(m)
                steps_II[m,0]=steps_I[m-1,0]+tri_mat[l,0]
                steps_I[:,:]=steps_II[:,:]
            else:
                var_1=steps_I[m-1,:]
                var_2=steps_I[m,:]
                var_new=np.hstack((var_1[var_1!=np.inf],var_2[var_2!=np.inf]))
                len_var=np.shape(var_new)[0]
                steps_II[m,0:len_var]=var_new+tri_mat[l,m]
                steps_I[:,:]=steps_II[:,:]
    print(steps_I[:,0:10])
    var_new=steps_I[steps_I!=np.inf]
    len_var=np.shape(var_new)[0]
    #print(len_var)
    det_steps[l,0:len_var]=var_new

[[91. inf inf inf inf inf inf inf inf inf]
 [inf inf inf inf inf inf inf inf inf inf]
 [inf inf inf inf inf inf inf inf inf inf]
 [inf inf inf inf inf inf inf inf inf inf]
 [inf inf inf inf inf inf inf inf inf inf]
 [inf inf inf inf inf inf inf inf inf inf]
 [inf inf inf inf inf inf inf inf inf inf]
 [inf inf inf inf inf inf inf inf inf inf]
 [inf inf inf inf inf inf inf inf inf inf]
 [inf inf inf inf inf inf inf inf inf inf]]
[[182.  inf  inf  inf  inf  inf  inf  inf  inf  inf]
 [152.  inf  inf  inf  inf  inf  inf  inf  inf  inf]
 [ inf  inf  inf  inf  inf  inf  inf  inf  inf  inf]
 [ inf  inf  inf  inf  inf  inf  inf  inf  inf  inf]
 [ inf  inf  inf  inf  inf  inf  inf  inf  inf  inf]
 [ inf  inf  inf  inf  inf  inf  inf  inf  inf  inf]
 [ inf  inf  inf  inf  inf  inf  inf  inf  inf  inf]
 [ inf  inf  inf  inf  inf  inf  inf  inf  inf  inf]
 [ inf  inf  inf  inf  inf  inf  inf  inf  inf  inf]
 [ inf  inf  inf  inf  inf  inf  inf  inf  inf  inf]]
2
[[192.  inf  inf  inf  inf  inf  inf

ValueError: could not broadcast input array from shape (626,) into shape (512,)

In [116]:
a=np.array([[1,2,3,4,np.inf,np.inf],[1,2,3,4,np.inf,np.inf]])
a[a!=np.inf]

array([1., 2., 3., 4., 1., 2., 3., 4.])